# Article...

## Setup

In [ ]:
!git clone https://github.com/NesyaLab/qaoa/

In [1]:
import os
os.chdir("/Users/main/Desktop/qaoa")

In [2]:
#!pip install -r requirements.txt

In [3]:
from config import *

In [4]:
from classes import Problems as P
from classes import Qaoa as Q
from functions import qaoa_utilities as qaoa_utils
from functions import maxcut_utilities as mcut_utils
from functions import qaoa_optimizers as optims
from functions import symmetry_utilities as sym_utils

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import sys

## Get data

In [6]:
graphs = []
for i in range(num_graphs):
    graph_path =  "/graph_"+str(i)+".nx"
    with open("data/graph_"+str(i)+".nx", 'rb') as f:
        g = pickle.load(f)
    graphs.append(g)

## Results

In [7]:
!mkdir results
results_path = "/results"

mkdir: results: File exists


### Brute force

In [8]:
bfs = [0]*len(graphs)
idx = 0
for graph in graphs:
    bfs[idx] = mcut_utils.compute_max_cut_exactly(graph)
    idx += 1

with open("results/brute_force_solution", 'wb') as file:
    pickle.dump(bfs, file)

### Qaoa vanilla

In [ ]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_vanilla_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            qaoa = Q.Qaoa(p=layer, G=G, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA extended with one "shadow" node

In [ ]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_extended_1_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            N = graphs[idx].get_number_of_nodes()
            G_ = graphs[idx].get_graph().copy()
            G_.add_node(N)
            extended_graph_instance = P.Problems(p_type="custom", G=G_)
            qaoa = Q.Qaoa(p=layer, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA extended with two "shadow" nodes

In [ ]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_extended_2_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            N = graphs[idx].get_number_of_nodes()
            G_ = graphs[idx].get_graph().copy()
            G_.add_node(N)
            G_.add_node(N+1)
            extended_graph_instance = P.Problems(p_type="custom", G=G_)
            qaoa = Q.Qaoa(p=layer, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA extended with one pendent edge

In [ ]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_extended_3_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            N = graphs[idx].get_number_of_nodes()
            G_ = graphs[idx].get_graph().copy()
            G_.add_node(N)
            u = list(G_.nodes())[N]
            v = list(G_.nodes())[random.randint(0,N-1)]
            G_.add_edge(u,v)
            extended_graph_instance = P.Problems(p_type="custom", G=G_)
            qaoa = Q.Qaoa(p=layer, G=extended_graph_instance, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
            x, _ = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            qaoa = Q.Qaoa(p=layer, G=graphs[idx], betas=x[:layer], gammas=x[layer:], mixer=mixer, seed=seed_, verbose=verbose)
            _, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
            AR = -f/bfs[idx]
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,AR), file)

### QAOA reduced

In [9]:
for seed_ in range(seed, seed+3):
    for layer in range(1,p+1):
        betas = qaoa_utils.generate_parameters(n=layer, k=1, seed=seed_)
        gammas = qaoa_utils.generate_parameters(n=layer, k=2, seed=seed_)
        for idx in range(num_graphs):
            name = "qaoa_reduced_graph_" + str(idx) + "_p_" + str(layer) + "_seed_" + str(seed_)
            G = graphs[idx]
            G_ = graphs[idx].get_graph().copy()
            G_prime = P.Problems(p_type="custom", G=G_)
            opt = [0]
            for edge in G_prime.get_edges():
                G_reduced = G_prime.get_graph().copy()
                G_reduced.remove_edges_from([edge])
                reduced_graph_instance = P.Problems(p_type="custom", G=G_reduced)
                qaoa = Q.Qaoa(p=layer, G=G, betas=betas, gammas=gammas, mixer=mixer, seed=seed_, verbose=verbose)
                x, f = optims.simple_optimization(qaoa, method=method, seed=seed_, verbose=verbose)
                AR = -f/bfs[idx]
                opt.append(AR)
            with open("results/" + name, 'wb') as file:
                pickle.dump((x,f,max(opt)), file)

### Spectrum

In [ ]:
# CALCULATING SPECTRAL RADIUS METRIC FOR EACH GRAPH

idx = 0
metric = {}
spectral_radii = []
avg_degrees = []
for graph in graphs:
    # The max eigenvalue is real
    rho = float(max(graph.get_adjacency_spectrum()))
    D = np.diag(np.sum(graph.get_adjacency_matrix(), axis=1))
    spectral_radii.append(rho)
    avg_degrees.append(int(np.sum(D)/D.shape[0]))
    metric[idx] = [avg_degrees[idx] / spectral_radii[idx]]
    idx += 1
with open("results/spectral_radii", 'wb') as file:
    pickle.dump(spectral_radii, file)
with open("results/spectral_metric_unperturbed", 'wb') as file:
    pickle.dump(metric, file)

In [ ]:
# CALCULATING SPECTRAL RADIUS METRIC FOR EACH GRAPH PERTURBATION

# One shadow node
idx = 0
metric_shadow1 = {}
spectral_radii_shadow1 = []
avg_degrees_shadow1 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    instance = P.Problems(p_type="custom", G=G)
    rho = float(max(instance.get_adjacency_spectrum()))
    D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
    spectral_radii_shadow1.append(rho)
    avg_degrees_shadow1.append(int(np.sum(D)/D.shape[0]))
    metric_shadow1[idx] = [avg_degrees_shadow1[idx] / spectral_radii_shadow1[idx]]
    idx += 1
with open("results/spectral_radii_shadow1", 'wb') as file:
    pickle.dump(spectral_radii_shadow1, file)
with open("results/spectral_metric_shadow1", 'wb') as file:
    pickle.dump(metric_shadow1, file)

# Two shadow nodes
idx = 0
metric_shadow2 = {}
spectral_radii_shadow2 = []
avg_degrees_shadow2 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    G.add_node(N+1)
    instance = P.Problems(p_type="custom", G=G)
    rho = float(max(instance.get_adjacency_spectrum()))
    D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
    spectral_radii_shadow2.append(rho)
    avg_degrees_shadow2.append(int(np.sum(D)/D.shape[0]))
    metric_shadow2[idx] = [avg_degrees_shadow2[idx] / spectral_radii_shadow2[idx]]
    idx += 1
with open("results/spectral_radii_shadow2", 'wb') as file:
    pickle.dump(spectral_radii_shadow2, file)
with open("results/spectral_metric_shadow2", 'wb') as file:
    pickle.dump(metric_shadow2, file)


# One pendent edge
idx = 0
metric_pendent = {}
spectral_radii_pendent = []
avg_degrees_pendent = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    u = list(G.nodes())[N]
    v = list(G.nodes())[random.randint(0,N-2)]
    G.add_edge(u,v)
    instance = P.Problems(p_type="custom", G=G)
    rho = float(max(instance.get_adjacency_spectrum()))
    D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
    spectral_radii_pendent.append(rho)
    avg_degrees_pendent.append(int(np.sum(D)/D.shape[0]))
    metric_pendent[idx] = [avg_degrees_pendent[idx] / spectral_radii_pendent[idx]]
    idx += 1
with open("results/spectral_radii_pendent", 'wb') as file:
    pickle.dump(spectral_radii_pendent, file)
with open("results/spectral_metric_pendent", 'wb') as file:
    pickle.dump(metric_pendent, file)

# One deleted edge
idx = 0
metric_deleted = {}
spectral_radii_deleted = {}
avg_degrees_deleted = {}
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G_ = P.Problems(p_type="custom", G=G)
    metric_deleted_vec = []
    avg_degrees_deleted_vec = []
    spectral_radii_deleted_vec = []
    for edge in G_.get_edges():
        G_reduced = G_.get_graph().copy()
        G_reduced.remove_edges_from([edge])
        instance = P.Problems(p_type="custom", G=G_reduced)
        rho = float(max(instance.get_adjacency_spectrum()))
        D = np.diag(np.sum(instance.get_adjacency_matrix(), axis=1))
        spectral_radii_deleted_vec.append(rho)
        avg_degrees_deleted_vec.append(int(np.sum(D)/D.shape[0]))
    spectral_radii_deleted[idx] = max(spectral_radii_deleted_vec)
    avg_degrees_deleted[idx] = max(avg_degrees_deleted_vec)
    metric_deleted[idx] = [avg_degrees_deleted[idx] / spectral_radii_deleted[idx]]
    idx += 1
with open("results/spectral_radii_deleted", 'wb') as file:
    pickle.dump(spectral_radii_deleted, file)
with open("results/spectral_metric_deleted", 'wb') as file:
    pickle.dump(metric_deleted, file)

In [ ]:
# CHECK

print(metric, "\n")
print(metric_shadow1, "\n")
print(metric_shadow2, "\n")
print(metric_pendent, "\n")
print(metric_deleted)

### Symmetries

In [ ]:
# CALCULATING |AUT(G)| FOR EACH GRAPH

num_symmetries = []
generating_symmetries = []
for graph in graphs:
    adjacency_dict = graph.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=graph, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=graph, adjacency_dict=adjacency_dict)
    num_symmetries.append(num_automorphisms)
    generating_symmetries.append(generators)
with open("results/aut_G", 'wb') as file:
      pickle.dump((num_symmetries,generating_symmetries), file)

In [ ]:
# CALCULATING |AUT(G')| FOR EACH GRAPH PERTURBATION

# One shadow node
num_symmetries_shadow1 = []
generating_symmetries_shadow1 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    instance = P.Problems(p_type="custom", G=G)
    adjacency_dict = instance.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
    num_symmetries_shadow1.append(num_automorphisms)
    generating_symmetries_shadow1.append(generators)


# Two shadow nodes
num_symmetries_shadow2 = []
generating_symmetries_shadow2 = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    G.add_node(N+1)
    instance = P.Problems(p_type="custom", G=G)
    adjacency_dict = instance.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
    num_symmetries_shadow2.append(num_automorphisms)
    generating_symmetries_shadow2.append(generators)


# One pendent edge
num_symmetries_pendent = []
generating_symmetries_pendent = []
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G.add_node(N)
    u = list(G.nodes())[N]
    v = list(G.nodes())[random.randint(0,N-2)]
    G.add_edge(u,v)
    instance = P.Problems(p_type="custom", G=G)
    adjacency_dict = instance.get_adjacency_dict()
    num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
    generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
    num_symmetries_pendent.append(num_automorphisms)
    generating_symmetries_pendent.append(generators)


# One deleted edge
idx = 0
num_symmetries_deleted = {}
generating_symmetries_deleted= {}
for graph in graphs:
    N = graph.get_number_of_nodes()
    G = graph.get_graph().copy()
    G_ = P.Problems(p_type="custom", G=G)
    num_symmetries_deleted_vec = []
    generating_symmetries_deleted_vec = []
    for edge in G_.get_edges():
        G_reduced = G_.get_graph().copy()
        G_reduced.remove_edges_from([edge])
        instance = P.Problems(p_type="custom", G=G_reduced)
        adjacency_dict = instance.get_adjacency_dict()
        num_automorphisms = sym_utils.get_number_of_automorphisms(G=instance, adjacency_dict=adjacency_dict)
        generators = sym_utils.get_symmetry_generators(G=instance, adjacency_dict=adjacency_dict)
        num_symmetries_deleted_vec.append(num_automorphisms)
        generating_symmetries_deleted_vec.append(generators)
    num_symmetries_deleted[idx] = num_symmetries_deleted_vec
    generating_symmetries_deleted[idx] = generating_symmetries_deleted_vec
    idx += 1

In [ ]:
# CHECK

print(num_symmetries)
print(num_symmetries_shadow1)
print(num_symmetries_shadow2)
print(num_symmetries_pendent)
print(num_symmetries_deleted)